In [13]:
import requests
import pandas as pd
from io import StringIO

In [7]:
URL = "https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/1/0/0/SEGMENTO"

# Cabeceras tipo navegador (las sec-*, gpc, etc. no son necesarias para requests)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                  " AppleWebKit/537.36 (KHTML, like Gecko)"
                  " Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "es-PE,es;q=0.9,en;q=0.8",
    "Referer": "https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/catalogo/listado",
}

resp = requests.get(URL, headers=HEADERS, timeout=30)
resp.raise_for_status()

# Extrae todas las tablas del HTML y localiza la que tiene columnas 'Codigo' y 'Titulo'
html_io = StringIO(resp.text)
tables = pd.read_html(html_io)
df_segmento = next(t for t in tables if {'Codigo', 'Titulo'}.issubset(set(t.columns)))

# Limpieza y selección de columnas
df_segmento = df_segmento[['Codigo', 'Titulo']].dropna().reset_index(drop=True).rename(columns={'Codigo': 'codigo_segmento', 'Titulo': 'segmento'})


In [12]:
lista = []
for i in df_segmento[29:].codigo_segmento:
    URL = f"https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/2/{i}/0/FAMILIA"

    # Cabeceras tipo navegador (las sec-*, gpc, etc. no son necesarias para requests)
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                    " AppleWebKit/537.36 (KHTML, like Gecko)"
                    " Chrome/120.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "es-PE,es;q=0.9,en;q=0.8",
        "Referer": "https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/catalogo/listado",
    }

    resp = requests.get(URL, headers=HEADERS, timeout=30)
    resp.raise_for_status()

    # Extrae todas las tablas del HTML y localiza la que tiene columnas 'Codigo' y 'Titulo'
    df_familia = pd.DataFrame()
    html_io = StringIO(resp.text) 
    tables = pd.read_html(html_io)
    try:
        df_familia = next(t for t in tables if {'Codigo', 'Titulo'}.issubset(set(t.columns)))
    except:
        print(f'Error en la URL: {URL}')
    # Limpieza y selección de columnas
    df_familia = df_familia[['Codigo', 'Titulo']].dropna().reset_index(drop=True).rename(columns={'Codigo': 'codigo_familia', 'Titulo': 'familia'})
    df_familia = df_segmento[df_segmento['codigo_segmento']==i].join(df_familia, how='cross')
    
    for j in df_familia.codigo_familia:
        URL = f'https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/3/{j}/0/CLASE'
        
        # Cabeceras tipo navegador (las sec-*, gpc, etc. no son necesarias para requests)
        HEADERS = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                        " AppleWebKit/537.36 (KHTML, like Gecko)"
                        " Chrome/120.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Accept-Language": "es-PE,es;q=0.9,en;q=0.8",
            "Referer": "https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/catalogo/listado",
        }

        resp = requests.get(URL, headers=HEADERS, timeout=30)
        resp.raise_for_status()

        # Extrae todas las tablas del HTML y localiza la que tiene columnas 'Codigo' y 'Titulo'
        df_clase = pd.DataFrame()
        html_io = StringIO(resp.text) 
        tables = pd.read_html(html_io)
        try:
            # Limpieza y selección de columnas
            df_clase = next(t for t in tables if {'Codigo', 'Titulo'}.issubset(set(t.columns)))
            df_clase = df_clase[['Codigo', 'Titulo']].dropna().reset_index(drop=True).rename(columns={'Codigo': 'codigo_clase', 'Titulo': 'clase'})
            df_clase = df_familia[df_familia['codigo_familia']==j].join(df_clase, how='cross')
        except:
            print(f'Error en la URL: {URL}')
            break
        
        for k in df_clase.codigo_clase:
            URL = f'https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/{k}/0/ITEM'
            
            # Cabeceras tipo navegador (las sec-*, gpc, etc. no son necesarias para requests)
            HEADERS = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                            " AppleWebKit/537.36 (KHTML, like Gecko)"
                            " Chrome/120.0.0.0 Safari/537.36",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "Accept-Language": "es-PE,es;q=0.9,en;q=0.8",
                "Referer": "https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/catalogo/listado",
            }

            resp = requests.get(URL, headers=HEADERS, timeout=30)
            resp.raise_for_status()

            # Extrae todas las tablas del HTML y localiza la que tiene columnas 'Codigo' y 'Titulo'
            df_commodity = pd.DataFrame()
            html_io = StringIO(resp.text) 
            tables = pd.read_html(html_io)
            try:
                df_commodity = next(t for t in tables if {'Codigo', 'Titulo'}.issubset(set(t.columns)))
                df_commodity = df_commodity[['Codigo', 'Titulo']].dropna().reset_index(drop=True).rename(columns={'Codigo': 'codigo_commodity', 'Titulo': 'commodity'})
            except:
                pd.DataFrame({'codigo_commodity': [f'{k}00'], 'commodity': ['sin commodity']})
                print(f'Error en la URL: {URL}')
            # Limpieza y selección de columnas
            df_commodity = df_clase[df_clase['codigo_clase']==k].join(df_commodity, how='cross')
            lista.append(df_commodity)

Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/511021/0/ITEM
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/3/5125/0/CLASE
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/531033/0/ITEM
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/570302/0/ITEM
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/761218/0/ITEM
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/855165/0/ITEM
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/855166/0/ITEM
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seace/api/cubso/buscador/modalBuscador2/4/855167/0/ITEM
Error en la URL: https://prod2.seace.gob.pe/buscador-publico-seac

In [ ]:
pd.concat(lista).reset_index(drop=True).to_csv('../data/processed/D_cubso.csv', index=False, sep='|')

In [14]:
df = pd.read_csv('../data/processed/D_cubso.csv', sep='|')

In [16]:
df

,codigo_segmento,segmento,codigo_familia,familia,codigo_clase,clase,codigo_commodity,commodity
0,10,"Material, accesorios y suministros de plantas ...",1010,Animales vivos,101015,Ganado,10101501.0,Gatos
1,10,"Material, accesorios y suministros de plantas ...",1010,Animales vivos,101015,Ganado,10101502.0,Perros
2,10,"Material, accesorios y suministros de plantas ...",1010,Animales vivos,101015,Ganado,10101504.0,Visón
3,10,"Material, accesorios y suministros de plantas ...",1010,Animales vivos,101015,Ganado,10101505.0,Ratas
4,10,"Material, accesorios y suministros de plantas ...",1010,Animales vivos,101015,Ganado,10101506.0,Caballos
...,...,...,...,...,...,...,...,...
197060,95,"Terreno, edificios, estructuras y vías princip...",9514,Edificios y estructuras prefabricadas,951418,Edificios y estructuras prefabricadas para ali...,95141805.0,Sistema de suelo para unidad de almacenamiento...
197061,95,"Terreno, edificios, estructuras y vías princip...",9514,Edificios y estructuras prefabricadas,951419,Edificios y estructuras médicas prefabricadas,95141901.0,Unidad médica
197062,95,"Terreno, edificios, estructuras y vías princip...",9514,Edificios y estructuras prefabricadas,951419,Edificios y estructuras médicas prefabricadas,95141902.0,Unidad de laboratorio
197063,95,"Terreno, edificios, estructuras y vías princip...",9514,Edificios y estructuras prefabricadas,951419,Edificios y estructuras médicas prefabricadas,95141903.0,Unidad dental


In [52]:
import time
import random
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://prod2.seace.gob.pe"
LISTADO_URL = f"{BASE}/buscador-publico-seace/api/cubso/catalogo/listado"
REFERER = LISTADO_URL

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                  " AppleWebKit/537.36 (KHTML, like Gecko)"
                  " Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "es-PE,es;q=0.9,en;q=0.8",
    "Referer": REFERER,
    "Origin": BASE,
}

def post_listado_por_commodity(session, cat4, titulo_filtro=""):
    """
    Envía el POST y devuelve el HTML cuando detecta al menos una fila con patrón de item.
    No depende de 'class="datatable"'.
    """
    data = [
        ("idPadre", str(cat4)),
        ("titulo", titulo_filtro),
        # token dummy (si el backend no valida recaptcha en server-side)
        ("token", "x_dummy_token"),
        # Rellenos opcionales (puedes omitirlos si quieres)
        ("descripcion", "SEGMENTO"),
        ("descripcion", "FAMILIA"),
        ("descripcion", "CLASE"),
        ("descripcion", "COMMODITY"),
        ("descripcion", "ITEM"),
        ("cat1", ""),
        ("cat2", ""),
        ("cat3", ""),
        ("cat4", str(cat4)),
        ("cat5", ""),
        ("titulo1", ""),
        ("titulo2", ""),
        ("titulo3", ""),
        ("titulo4", ""),
        ("titulo5", titulo_filtro),
    ]

    resp = session.post(LISTADO_URL, headers=HEADERS, data=data, timeout=40)
    if resp.status_code != 200:
        return None

    html = resp.text

    # Validación robusta 1: ¿existe patrón de '8 dígitos - 8 dígitos' en los links de item?
    if re.search(r'\b\d{8}-\d{8}\b', html):
        return html

    # Validación robusta 2: ¿existe una tabla que parezca la de resultados?
    soup = BeautifulSoup(html, "html.parser")
    candidate_tables = soup.select("table.table.table-hover.table-bordered")  # amplio
    for tb in candidate_tables:
        # ¿Tiene encabezados esperados?
        head_text = " ".join(h.get_text(strip=True) for h in tb.select("thead th"))
        expected = ("SEGMENTO" in head_text and "FAMILIA" in head_text and 
                    "CLASE" in head_text and "COMMODITY" in head_text and "ITEM" in head_text)
        # ¿Tiene al menos una fila en tbody?
        has_rows = bool(tb.select("tbody tr"))
        if expected and has_rows:
            return html

    # Si nada calza, no lo tomamos como éxito
    return None

def parse_items_desde_html(html):
    """
    Extrae items como:
    [{'codigo_commodity':'10101601','codigo_item':'00211181','item':'GALLINA DE POSTURA'}, ...]
    """
    soup = BeautifulSoup(html, "html.parser")
    out = []

    # Encuentra cualquier tabla de resultados “larga”; si hay varias, procesamos todas.
    tables = soup.select("table")
    for table in tables:
        # Revisa si el encabezado tiene columnas esperadas (para filtrar)
        thead = table.find("thead")
        if not thead:
            continue
        th_txt = " ".join(th.get_text(strip=True).upper() for th in thead.find_all("th"))
        if not all(x in th_txt for x in ["SEGMENTO", "FAMILIA", "CLASE", "COMMODITY", "ITEM"]):
            continue

        # Recorre filas del tbody
        tbody = table.find("tbody")
        if not tbody:
            continue

        for tr in tbody.find_all("tr", recursive=False):
            tds = tr.find_all("td", recursive=False)
            if len(tds) < 5:
                continue

            td_item = tds[4]

            # Busca el patrón 8d-8d en cualquier texto dentro de la celda item
            full_text = td_item.get_text(" ", strip=True)
            m = re.search(r'\b(\d{8})-(\d{8})\b', full_text)
            if not m:
                continue

            codigo_commodity, codigo_item = m.group(1), m.group(2)

            # Descripción: generalmente es el segundo <td> de la tabla interna
            item_desc = ""
            inner_tr = td_item.select_one("table tr")
            if inner_tr:
                inner_tds = inner_tr.find_all("td", recursive=False)
                if len(inner_tds) >= 2:
                    item_desc = inner_tds[1].get_text(strip=True)
            if not item_desc:
                # Fallback: tomar el último texto no vacío después del código
                texts = [t.strip() for t in td_item.stripped_strings if t.strip()]
                # Si encontramos el string '10101601-00211181', la desc suele estar después
                try:
                    idx = next(i for i, t in enumerate(texts) if re.fullmatch(r'\d{8}-\d{8}', t))
                    if idx + 1 < len(texts):
                        item_desc = texts[idx + 1]
                except StopIteration:
                    # Último fallback
                    item_desc = texts[-1] if texts else ""

            out.append({
                "codigo_commodity": codigo_commodity,
                "codigo_item": codigo_item,
                "item": item_desc
            })

    return out

def scrape_items_para_lista_de_commodities(df_commodities, pausa=(0.8, 2.0)):
    """
    df_commodities: DF que contenga al menos col 'codigo_commodity' y, si quieres,
                    las columnas de contexto (segmento/familia/clase/commodity).
    Devuelve un DF con las columnas:
      ['codigo_segmento','segmento','codigo_familia','familia','codigo_clase','clase',
       'codigo_commodity','commodity','codigo_item','item','codigo_cubso']
    """
    session = requests.Session()
    resultados = []

    # Si tienes muchas filas, considera cachear e ir guardando en CSV incrementalmente
    for idx, row in df_commodities.iterrows():
        cat4 = str(row["codigo_commodity"])
        # Pequeña pausa aleatoria para no golpear el sitio
        time.sleep(random.uniform(*pausa))

        html = post_listado_por_commodity(session, cat4)
        if not html:
            print(f"[WARN] No se pudo obtener listado para commodity {cat4}. Intentar luego o usar Selenium.")
            continue

        items = parse_items_desde_html(html)
        if not items:
            print(f"[INFO] Commodity {cat4} sin items o no parseable.")
            continue

        # Agrega contexto de categorías si lo tienes en df_commodities
        for it in items:
            fila = {
                "codigo_segmento": row.get("codigo_segmento"),
                "segmento": row.get("segmento"),
                "codigo_familia": row.get("codigo_familia"),
                "familia": row.get("familia"),
                "codigo_clase": row.get("codigo_clase"),
                "clase": row.get("clase"),
                "codigo_commodity": it["codigo_commodity"],
                "commodity": row.get("commodity"),
                "codigo_item": it["codigo_item"],
                "item": it["item"],
            }
            fila["codigo_cubso"] = f"{fila['codigo_commodity']}-{fila['codigo_item']}"
            resultados.append(fila)

    return pd.DataFrame(resultados)

In [27]:
df.codigo_commodity = df.codigo_commodity.astype(int)

In [31]:
df[:1]

,codigo_segmento,segmento,codigo_familia,familia,codigo_clase,clase,codigo_commodity,commodity
0,10,"Material, accesorios y suministros de plantas ...",1010,Animales vivos,101015,Ganado,10101501,Gatos


In [60]:
session = requests.Session()
resultados = []

# Si tienes muchas filas, considera cachear e ir guardando en CSV incrementalmente
for idx, row in df[df.codigo_commodity==10101601].iterrows():
    cat4 = str(row["codigo_commodity"])
    # Pequeña pausa aleatoria para no golpear el sitio
    time.sleep(random.uniform(0.8, 2.0))

    html = post_listado_por_commodity(session, cat4)

In [61]:
html

'<!DOCTYPE html>\r\n<html xmls:th="http:www.thymeleaf.org">\r\n\r\n<head>\r\n\t\r\n\t<!-- SD-OTI-2025-029 Implementacion de Google Analytics --> \r\n\t<script async src="https://www.googletagmanager.com/gtag/js?id=G-KW833NE2ZY"></script> \r\n\t<script> \r\n\t\twindow.dataLayer = window.dataLayer || []; \r\n\t\tfunction gtag(){\r\n\t\t\tdataLayer.push(arguments);\r\n\t\t} gtag(\'js\', new Date()); \r\n\t\t\r\n\t\tgtag(\'config\', \'G-KW833NE2ZY\'); \r\n\t</script> \r\n\t<!-- Fin SD-OTI-2025-029--> \r\n<meta charset="UTF-8"  />\r\n<link rel="shortcut icon" href="/buscador-publico-seace/images/lupaColor.ico" type="image/vnd.microsoft.icon" />\r\n<!-- Bootstrap CSS -->\r\n<link\r\n\thref="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css"\r\n\trel="stylesheet"\r\n\tintegrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3"\r\n\tcrossorigin="anonymous">\r\n<!-- Bootstrap Font Icon CSS -->\r\n<link rel="stylesheet"\r\n\thref="https://cdn.jsdeliv

In [59]:
df[df.codigo_commodity==10101601]

,codigo_segmento,segmento,codigo_familia,familia,codigo_clase,clase,codigo_commodity,commodity
19,10,"Material, accesorios y suministros de plantas ...",1010,Animales vivos,101016,Pájaros y aves de corral,10101601,Pollos vivos


In [62]:
# Suponiendo df_commodity tiene al menos:
# ['codigo_segmento','segmento','codigo_familia','familia','codigo_clase','clase','codigo_commodity','commodity']
df_items = scrape_items_para_lista_de_commodities(df[df.codigo_commodity==10101601])
# df_items.to_csv("cubso_items.csv", index=False, encoding="utf-8-sig")